In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
from functools import reduce
from difflib import SequenceMatcher
from math import isclose

def create_ratio_name(x, key, ratio_label="ratio_name"):
    x[ratio_label] = SequenceMatcher(None, x.name.value, key).ratio()
    return x

def create_ratio_date(x, key, ratio_label="ratio_date"):
    try:
        year = int(x.yearpublished.value.strip())
    except:
        year = 0
    x[ratio_label] = year/key
    if x[ratio_label] > 1:
        x[ratio_label] = 1/x[ratio_label]
    return x

def sort_by_closest(l, key, create_ratio_field=create_ratio_name):
    l = list(map(lambda x: create_ratio_field(x, key), l))
    l.sort(reverse=True,key=lambda x: x.ratio_name)
    return l

def pick_closest(x, y):
    ratio_name = "ratio_name"
    ratio_date = "ratio_date"
    
    if x[ratio_name] > y[ratio_name]:
        return x

    if isclose(x[ratio_name], y[ratio_name]):
        return x if x[ratio_date] > y[ratio_date] else y

    return y

def get_closest_match(name, date):
    l = conn.search(name)
    hits = int(l["items"].total)
    
    if hits == 0:
        return {'id': "not_found"}
    elif hits > 1:
        l = l['items'].item
        l = map(lambda x: create_ratio_name(x, name), l)
        l = map(lambda x: create_ratio_date(x, date), l)
        match = reduce(pick_closest, l)
        return match

    return l['items'].item


In [88]:
import json
from libbgg.apiv1 import BGG
# You can also use version 2 of the api:
from libbgg.apiv2 import BGG as BGG2

conn = BGG2()
game_name = "7 wonders"
realease_date = 2020
# conn.search(game_name)
# get_closest_match(game_name, realease_date)


In [95]:
import Ludopedia
from tqdm import tqdm

collection_name = 'scaroni'
game_type = "base"
parser = Ludopedia.CollectionParser()
parser.fetch_collection(collection_name, tipo='colecao', tipo_jogo=game_type)    
gameParser = Ludopedia.GamePageParser()

def get_bgg_info(g):
    game_title = g['title'].replace(':', '').replace('!', '')
    results = conn.search(game_title)
    if "boardgame" not in results.boardgames:
        g['bgg_id'] = "not found"
    elif type(results.boardgames.boardgame) is list:
        g['bgg_id'] = sort_by_closest(results.boardgames.boardgame, game_title)[0]['objectid']
    else:
        g['bgg_id'] = results.boardgames.boardgame.objectid
    return g

def get_ludopedia_info(g):
    info = gameParser.parse_game_page(g['link'])
    for i in info:
        g[i] = info[i]
    
    return g

def fetch_all_info(g):
    g = get_ludopedia_info(g)
    game_title = g['title'].replace(':', '').replace('!', '')
    g['bgg_id'] = get_closest_match(game_title, g['release_date'])['id']
    
    return g

# collection = list(map(get_bgg_info, tqdm(parser.collection)))
# collection = list(map(get_ludopedia_info, tqdm(collection)))
collection = list(map(fetch_all_info, tqdm(parser.collection)))

 82%|████████▏ | 49/60 [01:35<00:21,  1.94s/it]


HTTPError: HTTP Error 429: Too Many Requests

In [96]:
import pandas as pd

collection = pd.DataFrame(collection)
characteristics = pd.read_csv("csvs/game_escolheitor.csv")
characteristics.columns = ['title', 'players', 'experience', 'interaction', 'weight', "learning_curve", "duration"]
prices = pd.read_csv("csvs/precos.csv")
prices = prices[prices.columns[:-3]]
prices.columns = ['title', 'sleeved', 'how_many_cards', 'money_spent', 'obs']

collection_data = collection.join(characteristics.set_index('title'), on='title')
collection_data = collection_data.join(prices.set_index('title'), on='title')
collection_data

,link,title,owned,release_date,has,had,bgg_id,rating,description,players,experience,interaction,weight,learning_curve,duration,sleeved,how_many_cards,money_spent,obs
0,https://www.ludopedia.com.br/jogo/13-days-the-...,13 Days: The Cuban Missile Crisis,True,2015,35,4,177590,NaN,NaN,2,tática,indireta,leve,1 partida,30-45 min,Sim,NaN,70.0,sleeves padrão
1,https://www.ludopedia.com.br/jogo/7-wonders,7 Wonders,True,2010,4230,470,68448,8.0,Esse quanto mais eu jogo mais eu gosto. Cardga...,3-5,tática,baixa/nula,leve,2-3 partidas,30-45 min,Sim,NaN,180.0,sleeves padrão
2,https://www.ludopedia.com.br/jogo/academia,Academia,True,1989,401,51,8815,7.5,"Party game de palavras e criatividade, basicam...",3-6,social,indireta,leve,1 partida,30-45 min,NaN,NaN,0.0,NaN
3,https://www.ludopedia.com.br/jogo/bandido,Bandido,True,2016,1662,35,191925,7.5,Card game cooperativo de construção de caminho...,1-4,tática,coop,leve,1 partida,10-20 min,Não,NaN,35.0,NaN
4,https://www.ludopedia.com.br/jogo/bang-pocket,Bang! Pocket,True,2014,1059,160,not_found,7.0,"Party game de cartas com blefe, interação dire...",4-7,social,direta (semi-coop),leve,1 partida,30-45 min,Não,NaN,0.0,NaN
5,https://www.ludopedia.com.br/jogo/black-stories-2,Black Stories 2,True,2005,1125,95,20750,6.5,"Party game de charadas, não é exatamente um jo...",2+,social,coop,leve,1 partida,10-20 min,Não,NaN,40.0,NaN
6,https://www.ludopedia.com.br/jogo/brass-birmin...,Brass: Birmingham,True,2018,866,54,224517,9.5,Jogo econômico com construção de rotas e simul...,2-4,estratégica,indireta,médio-pesado,2-3 partidas,120-180 min,Sim,NaN,280.0,sleeves premium e caixinha organizadora extern...
7,https://www.ludopedia.com.br/jogo/hunt-for-the...,Busca pelo Anel,True,2017,0,17,216070,9.0,Jogo de movimentação secreta que simula o iníc...,2-5,imersiva,direta,médio,2-3 partidas,90-180 min,Sim,NaN,195.0,sleeves padrão e caixinha organizadora na caixa
8,https://www.ludopedia.com.br/jogo/carnival-zom...,Carnival Zombie (Segunda Edição),True,2019,27,0,not_found,S/N,"Pre venda, efetuada. Chegada estimada pro fim ...",NaN,imersiva,coop,médio,2-3 partidas,120-150 min,NaN,NaN,550.0,edição deluxe da pré-venda da funbox
9,https://www.ludopedia.com.br/jogo/claim,Claim,True,2017,446,21,233961,7.0,Jogo de vaza para 2 jogadores é um negócio mei...,2,tática,direta,leve,1 partida,10-20 min,Sim,NaN,20.0,sleeves padrão
